In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
import tensorflow as tf
import warnings
import tensorflow_hub as hub
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
warnings.filterwarnings('ignore')
from sklearn.metrics import pairwise_distances
from heapq import nsmallest
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
test_articles = pd.read_csv('../dataset/test_articles_newyork.csv', delimiter=',')

In [3]:
fixed_index = pd.read_csv('../dataset/fixed_test_set_newyork_max_recall5.csv', delimiter=',')

In [4]:
corpus = []

for i, row in tqdm(test_articles.iterrows()):
    
    corpus.append(row['article_title'])
    corpus.append(str(row['article_meta_description']))
    
for i, row in fixed_index.iterrows():
    
    corpus.append(row['table_page_title'])
    corpus.append(str(row['table_page_main_passage']))

vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

148it [00:00, 6042.72it/s]


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [5]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [6]:
def get_mrr(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    index_match = 1

    for idTable in ranked_tables_ID:
        
        if idTable[0] == ID_goal:
    
            accuracy = 1/index_match
            break;
        
        index_match = index_match + 1
   
    return accuracy

In [7]:
test_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   article_full_text         141 non-null    object
 1   article_key_match         148 non-null    object
 2   article_keywords          143 non-null    object
 3   article_main_passage      148 non-null    object
 4   article_meta_description  132 non-null    object
 5   article_meta_keywords     58 non-null     object
 6   article_summary           141 non-null    object
 7   article_tags              54 non-null     object
 8   article_title             148 non-null    object
 9   article_url               148 non-null    object
dtypes: object(10)
memory usage: 11.7+ KB


In [8]:
fixed_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14800 entries, 0 to 14799
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   label_index              14800 non-null  object
 1   table_page_id            14800 non-null  object
 2   table_page_title         14800 non-null  object
 3   table_page_main_passage  14151 non-null  object
 4   table_page_keywords      10971 non-null  object
dtypes: object(5)
memory usage: 578.2+ KB


In [9]:
evaluate_topk = [1,5,10,20,50]

result = []

#evaluating each topk value
for TOP_K in evaluate_topk:

    accuracy = []
    mrr = []

    for i, row in tqdm(test_articles.iterrows()):

        #current article values
        article_ID = row['article_key_match']
        article_title_text = str(row['article_title'])
        article_meta_description_text = str(row['article_meta_description'])

        #embedding and model variables
        article_title1 = []
        article_title2 = []
        title_table1 = []
        title_table2 = []
        ranked_tables_model = []

        #return index
        return_index = fixed_index.loc[fixed_index['label_index'] == row['article_key_match']]
        
        tfidf_vector1 = vectorizer.transform([article_title_text])
        tfidf_vector2 = vectorizer.transform([article_meta_description_text])
       
        article_title1.append(tfidf_vector1[0].toarray()[0])
        article_title2.append(tfidf_vector2[0].toarray()[0])

        #creating embedding 
        for i, row in return_index.iterrows():
            
            tfidf_vector3 = vectorizer.transform([str(row['table_page_title'])])
            tfidf_vector4 = vectorizer.transform([str(row['table_page_main_passage'])])
            
            title_table1.append(tfidf_vector3[0].toarray()[0])
            title_table2.append(tfidf_vector4[0].toarray()[0])
        
        distance_vector1 = pairwise_distances(article_title1, title_table1, metric='cosine')
        distance_vector2 = pairwise_distances(article_title2, title_table2, metric='cosine')
        final_vector = (np.array(distance_vector1) + np.array(distance_vector2)) / 2.0
        
        #creating the final dataframe
        for i in range(0,len(final_vector[0])):

            ranked_tables_model.append([return_index.iloc[i]['table_page_id'],return_index.iloc[i]['table_page_title'],final_vector[0][i]]) 

        data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
        data_frame_sorting = data_frame.sort_values('table_ranking')
        
        selected_top = data_frame_sorting.head(TOP_K)
#         max_score = selected_top['table_ranking'].max()
#         draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] <= max_score]
        final_ranked_tables = selected_top.iloc[:,0:1].values
    
#         print("")
#         print("query:" +article_title_text)
#         print("")
#         print(draw_tables_socres)

        #getting topk accuracy
        accuracy.append(get_accuracy(article_ID, final_ranked_tables))

        #testing mean reciprocal rank at k = 50
        if TOP_K == 50:

            mrr.append(get_mrr(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(TOP_K),str(round(np.mean(accuracy),4))])

print("TF-IDF METHOD")
print(result[0])
print(result[1])
print(result[2])
print(result[3])
print(result[4])
print("MRR: "+str(round(np.mean(mrr),4)) )
print(mrr)
print("")

148it [03:39,  1.48s/it]
148it [03:29,  1.41s/it]
148it [01:48,  1.37it/s]
148it [03:41,  1.50s/it]
148it [03:38,  1.48s/it]

TF-IDF METHOD
['Acc@1', '0.3514']
['Acc@5', '0.4865']
['Acc@10', '0.5541']
['Acc@20', '0.723']
['Acc@50', '0.8311']
MRR: 0.4241
[0, 0.023255813953488372, 1.0, 1.0, 0, 0, 0, 0.038461538461538464, 0.07142857142857142, 1.0, 0.5, 0, 0.3333333333333333, 1.0, 0.07692307692307693, 1.0, 0, 0.5, 1.0, 1.0, 0.034482758620689655, 0, 1.0, 1.0, 1.0, 0, 0.022727272727272728, 0.25, 0, 0.07142857142857142, 1.0, 0.1111111111111111, 0.3333333333333333, 0.021739130434782608, 0.3333333333333333, 0.0625, 0.07142857142857142, 0.06666666666666667, 0.2, 1.0, 0.07142857142857142, 0.3333333333333333, 0.14285714285714285, 1.0, 0.07142857142857142, 0, 1.0, 0.05, 0.045454545454545456, 1.0, 0.1111111111111111, 0.0625, 0.07142857142857142, 0.043478260869565216, 1.0, 0.3333333333333333, 0.3333333333333333, 1.0, 0.07142857142857142, 1.0, 1.0, 0.3333333333333333, 0.2, 0.14285714285714285, 1.0, 1.0, 0.07692307692307693, 1.0, 1.0, 1.0, 0.07142857142857142, 0.5, 0.14285714285714285, 0.043478260869565216, 0, 0.2, 0.07142857